In [ ]:
import mailbox
import email.utils
import pandas as pd
import exifread
from datetime import datetime, timezone
import pytz
from ipywidgets import IntProgress, IntText

In [ ]:
filename = "tgthorley.mbox"
filepath = './data/test/'

mbox = mailbox.mbox(filepath+ filename)

In [ ]:
#takes an image path and retruns exif data
def parse_img(imgpath):
    try:
        f = open(imgpath, 'rb')
        tags = exifread.process_file(f)
        return tags
    except:
        print("Failed to extract exif %s" % imgpath)
        return []

In [ ]:
#takes a message with an attached immage  and returns the exif data
def getexif_from_attached_image(message):
    for part in message.walk():
        if 'image' in part.get_content_type():
            try:
                filename = part.get_filename()
                filepath = "./attachements/"+filename
                fb = open(filepath,'wb')
                fb.write(part.get_payload(decode=True))
                fb.close()
                tags = parse_img(filepath)
                return tags
            except:
                print ("File not found", filename)
                continue
        else: continue

In [ ]:
emails = []
i=0
progressBar = IntProgress(description='Processing emails '+str(len(mbox)), min=0, max=len(mbox), style = {'description_width': 'initial'})
progressText = IntText(value=0, description='Processed so far: ', style = {'description_width': 'initial'})
display(progressBar, progressText)

#iterate through th
for message in mbox:
    i= i+1
    if message.get_content_maintype() == 'multipart':
        for part in message.walk():
            if 'image' in part.get_content_type():
                tags = getexif_from_attached_image(message)
                try:
                    if "EXIF DateTimeOriginal" in tags:
                        emaildate = email.utils.parsedate_to_datetime(message['Date']).astimezone(pytz.utc)
                        imgdate = datetime.strptime(tags["EXIF DateTimeOriginal"].values, '%Y:%m:%d %H:%M:%S').astimezone(pytz.utc)
                        timediff = emaildate - imgdate
                        if timediff.total_seconds() > 0 and timediff.total_seconds() < 86400:
                            print ('WARNING: Image taken less than a day after sending\n', message['Message-ID'], message['Subject'] )
                        email_sum = {
                            "email_date": emaildate,
                            "img_date": imgdate,
                            "_id": message['Message-ID'],
                            "subject": message['Subject'],
                            "timediff": timediff
                        }
                        emails.append(email_sum)
                        progressBar.value = i
                        progressText.value = i
                except: 
                    print("Error", message['Subject'])
                    continue

In [ ]:
df = pd.DataFrame(emails)
df.sort_values("timediff")